In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # device GPU if avaiable

In [8]:
# hyperparameters
num_epochs = 10
batch_size = 4
learning_rate = 0.01

In [4]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'house', 'ship', 'truck')

100%|██████████| 170498071/170498071 [00:13<00:00, 13029265.84it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [5]:
# Implementation of CNN
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.conv1 = nn.Conv2d(3, 6, 5) # 3 input channels (r,g,b), output channel size of 6, and kernel size of 5
    self.pool = nn.MaxPool2d(2, 2)
    self.conv2 = nn.Conv2d(6, 16, 5)
    self.fc1 = nn.Linear(16*5*5, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)
  
  def forward(self, x):
    # Convolution steps
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    
    # Flattening steps
    x = x.view(-1, 16*5*5) # first index is for number of samples we have in our batch (4 in this case)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x


In [6]:
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [9]:
n_total_steps = len(train_loader)

for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_loader):
    # original shape: [4, 3, 32, 32] = 4, 3, 1024
    # input_layer: 3 input channels, 6 output channels, 5 kernel size
    images = images.to(device)
    labels = labels.to(device)

    # forward pass
    outputs = model(images)
    loss = criterion(outputs, labels)

    # backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i+1) % 2000 == 0:
      print(f"Epoch {epoch+1}/{num_epochs}, Step {i+1}/{n_total_steps}, Loss {loss.item():.4f}")

    #print("Finished Training the Model")

# testing
with torch.no_grad():
  n_correct = 0
  n_samples = 0
  n_class_correct = [0 for i in range(10)]
  n_class_samples = [0 for i in range(10)]

  for images, labels in test_loader:
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)

    _, predicted = torch.max(outputs, 1)
    n_samples += labels.size(0)
    n_correct += (predicted == labels).sum().item()

    for i in range(batch_size):
      label = labels[i]
      pred = predicted[i]

      if(label==pred):
        n_class_correct[label] += 1
      
      n_class_samples[label] += 1

  acc = 100.0 * n_correct / n_samples
  print(f'Accuracy of the network: {acc} %')

  for i in range(10):
    acc = 100.0 * n_class_correct[i] / n_class_samples[i]
    print(f'Accuracy of {classes[i]}: {acc} %')


Epoch 1/10, Step 2000/12500, Loss 1.0617
Epoch 1/10, Step 4000/12500, Loss 1.4392
Epoch 1/10, Step 6000/12500, Loss 0.3187
Epoch 1/10, Step 8000/12500, Loss 0.0529
Epoch 1/10, Step 10000/12500, Loss 0.8881
Epoch 1/10, Step 12000/12500, Loss 0.9282
Epoch 2/10, Step 2000/12500, Loss 0.4553
Epoch 2/10, Step 4000/12500, Loss 0.5683
Epoch 2/10, Step 6000/12500, Loss 1.6721
Epoch 2/10, Step 8000/12500, Loss 1.3703
Epoch 2/10, Step 10000/12500, Loss 1.0654
Epoch 2/10, Step 12000/12500, Loss 0.6492
Epoch 3/10, Step 2000/12500, Loss 0.8814
Epoch 3/10, Step 4000/12500, Loss 0.5439
Epoch 3/10, Step 6000/12500, Loss 0.6229
Epoch 3/10, Step 8000/12500, Loss 1.3979
Epoch 3/10, Step 10000/12500, Loss 1.3146
Epoch 3/10, Step 12000/12500, Loss 0.1543
Epoch 4/10, Step 2000/12500, Loss 1.4861
Epoch 4/10, Step 4000/12500, Loss 0.2086
Epoch 4/10, Step 6000/12500, Loss 0.5325
Epoch 4/10, Step 8000/12500, Loss 1.7736
Epoch 4/10, Step 10000/12500, Loss 2.0964
Epoch 4/10, Step 12000/12500, Loss 0.1802
Epoch 5/